In [ ]:
import sys
import numpy as np
import pandas as pd
import patsy as pat
import statsmodels.api as smapi
import re
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

In [27]:
with open("discrete.dataformL", "r") as fh:
    line_list = []
    lines = fh.readlines()
    for i in range(0, len(lines)):
        line = lines[i]
        line_list.append(line)

    df = pd.DataFrame([sub.split("\t") for sub in line_list])

# Note: I've created a bunch of global variables here, but they could DEFINITELY be turned into a Class.
# This may increase security and robustness of the program ...
    
# Performing a VIF calculation in python via the method in statsmodels requires a constant (to be taken as intercept)
# (This may seem odd, but it turns out that defining the zero-order interaction to produce a column of all ones is very convenient -
# which means pat.dmatrix may do this for me. The hard part is defining the equation.

# Create DataFrame sans ID (with PHENO and outcome covariates): [VERIFIED]
    df_trim = df.iloc[:, 1:]
    trim_row_one = df_trim.iloc[0,:]
    df_trim.columns = trim_row_one
   # df_trim.drop('0')
    print(df_trim.axes)
# Create a DataFrame with only the predicited outcome of regression (phenotype): [VERIFIED]
#    df_pheno = df_trim.iloc[:, :1]
#    pheno_row_one = df_pheno.iloc[0,:]
#    df_pheno.columns = pheno_row_one
#    df_pheno.drop(df_pheno.iloc[0,:])
#    print(df_pheno.head())
# Create a DataFrame to represent only the predictors of regression (sans outcome): [VERIFIED]
#    df_predict = df_trim.iloc[:, 1:]
#    predict_row_one = df_predict.iloc[0,:]
#    df_predict.columns = predict_row_one
#    df_predict.drop(df_predict.iloc[0,:])
#    print(df_predict.head())
# Turn row 0 of the DataFrame into column labels - then remove row 0.
#    pred_labels = df_predict.columns
#    print(pred_labels)
# Create a list of labels from columns of new DataFrame:
#    all_labels = df_trim.columns
#    print(all_labels)
   # trim_labels = df_trim.columns
   # print(trim_labels)

[RangeIndex(start=0, stop=501, step=1), Index(['PHENO', 'SEX_COV', 'AGE', 'UPSIT', 'FAMILY_HISTORY', 'snp410_T',
       'snp403_T', 'snp164_C', 'snp439_G', 'snp370_A',
       ...
       'snp95_A', 'snp377_G', 'snp97_T', 'snp382_T', 'snp450_A', 'snp107_A',
       'snp224_C', 'snp368_G', 'snp304_C', 'snp306_G\n'],
      dtype='object', name=0, length=472)]


In [3]:
# This cell contains a list of functions/methods I'm trying to use to solve the patsy.dmatricies() problem. Some solutions
# will use them, others will not.

# Method 1: Compress the global list of columns from the DataFrame into string representing all  columns with interaction.
# Note that this method should be passed on lists without the "ID" column (I.E column [0] should be 'SEX_COV').
# Currently, global list 'pred_labels' satisfies this condition.

def coalesce_covariates(X):    
         predictors = ' : '.join(X)
         return predictors
coalesce_covariates(pred_labels)
predictor_list = coalesce_covariates(pred_labels)
trimmed_list = coalesce_covariates(trimmed_list)

print(trimmed_list)

features = ":".join(df_trim.columns - ["PHENO"])
print(features)

# print(predictor_list) [VERIFIED]

# Lets see if dmatricies will work with this list:

# y, X = pat.dmatrices('PHENO ~ snp304_C : snp306_G', df_trim, return_type='dataframe')

# Method 2: Given a DataFrame and a string representing 'interaction' of all columns of the DataFrame,
# call the vif() method with an SLR with 'PHENO' as the outcome variable.
# This is where the VIF calculation is breaking. Is this the thing where it needs a constant column?

#def call_vif(X, V):
#    dropped = True
#    while dropped:
#        dropped = False
#        all_multico = [vif(X.iloc[:, V].values, ix) for ix in range(X.iloc[:, V].shape[1])]
#        return all_multico
    
#call_vif(df_predict, predictor_list)    
    

# See above - I split this into separate defs to figure out where it was breaking.
#def calculate_vif(X, thresh=5.0):
#    variables = list(range(X.shape[1]))
#    dropped = True
#    while dropped:
        
        # Call .vif method on the iloc of every predictor variable in the dataframe.
        # ix is just a placeholder for the number of covariates.
#        dropped = False
#        colin = [vif(X.iloc[:, variables].values, ix) 
#                for ix in range(X.iloc[:, variables].shape[1])]

        # Set highest covariance to 'maxlock' and compare it to threshold.
        # Repeat this while loop until highest covariance is < threshold.
#        maxloc = colin.index(max(colin))
#        if max(colin) > thresh:
#            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
#                  '\' at index: ' + str(maxloc))
#            del variables[maxloc]
#            dropped = True

#    print('Remaining variables:')
#    print(X.columns[variables])
#    return X.iloc[:, variables]
# calculate_vif(df_predict, thresh=5.0)

NameError: name 'trimmed_list' is not defined

# Okay, so the following block is a huge mess for a reason.
# Recall that the goal is to get the .dmatrix or .dmatrices function working, and conduct a linear regression, given a DataFrame with X number of covariates.
# I've tried a couple of approaches to get the function to work.
#1: Tried to describe the column labels as a flattened string of numbers I.E - PHENO (1st label) versus EVERYTHING ELSE (separated by : to indicate interaction for all the other labels).
#2: Adjusting the first row of the input DataFrame - turn it into column labels (then remove the first row). From there, try the same thing - compare "PHENO" ~ "EVERYTHING ELSE (interacting).
PROBLEM: Having a hell of a time making the function read in properly.

In [ ]:
    # ATTEMPT 1):
    # Convert the list you made earlier into a string representing interaction of all of the predictors in dataset:
    
    #pred_num_list = list(range(2, pred_num))
    #s = [str(i) for i in pred_num_list]
    #num_list = ":".join(s)
    #print(num_list)
    
    # ATTEMPT 2):
    
    # Break into left and right hand side; y and X
    # Okay so some notes on patsy in relation to VIF score of all predictors:
    # Use the '~' to separate the two functions - the two sides to be compared.
    # Use '+' to add factors together
    # Use ':' to compare two factors together
    
    formula = ('PHENO ~ snp410_T,snp403_T').encode('ascii')
    print(formula)
    y, X = pat.dmatrices(formula, df_predict, return_type="dataframe")
    multicorr_table = pd.DataFrame()
    multicorr_table["VIF Factor"] = [vif(X.values, i) for i in range(X.shape[1])]
    multicorr_table["features"] = X.columns
    print(multicorr_table.round(1))
    
    # ATTEMPT 3):
    # Since this function really really wants to add columns together, could I write a method that returns a string 
    # representing the name of all the columns in the dataset added together?
    
    

#3: I've tried to input the first arugment of the .dmatrices function in several different ways, but have yet to find a workaround:
    a): 'PHENO ~' + pred_labels
    b): 'PHENO ~ + pred_labels'
    c): Starting by making the formula another variable I.E: ('PHENO ~' + pred_labels) or some variation and then:
        pat.dmatrices(formula, ...) but that hasn't worked.
    d): Various errors have been raised as I've played around with the formula - the most vexxing being "PatsyError: model is 
    missing required outcome variables." One resource suggested I try encoding the formula into 'ascii' - which turned out as:
    formula = ('PHENO ~ + pred_labels' + name).encode('ascii') but that hasn't worked either, same error (where 'name' is
    just the df.name).

In [ ]:
    # Remove the ID column (not used as a predictor) may not be necessary if patsy formula can exclude ID label itself.
   # vif_df.droplevel('0', axis=1)
   # print(vif_df)

   #Attempt #1 to conduct VIF analysis 
    
 #   VIF_list = pd.Series([vif(X.values, i) for i in range(X.shape[1])], index=X.columns)
    
 
 # print(VIF_list)
    

In [ ]:
# See above - I split this into separate defs to figure out where it was breaking.
def calculate_vif(X, thresh=5.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        
        # Call .vif method on the iloc of every predictor variable in the dataframe.
        # ix is just a placeholder for the number of covariates.
        dropped = False
        colin = [vif(X.iloc[:, variables].values, ix) 
                for ix in range(X.iloc[:, variables].shape[1])]

        # Set highest covariance to 'maxlock' and compare it to threshold.
        # Repeat this while loop until highest covariance is < threshold.
        maxloc = colin.index(max(colin))
        if max(colin) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]
# calculate_vif(df_predict, thresh=5.0)